In [ ]:
import pandas as pd
import numpy as np

# Định nghĩa tiêu đề cho dữ liệu
column_names = ["Id", "Name", "Age", "Weight", "m0006", "m0612", "m1218", "f0006", "f0612", "f1218"]

# Đọc dữ liệu CSV và thêm tiêu đề nếu thiếu
df = pd.read_csv("patient_heart_rate.csv", names=column_names)

# Hiển thị dữ liệu ban đầu
print("Dữ liệu ban đầu:")
print(df.head())

# 1. Tách cột Name thành Firstname và Lastname
df[['Firstname', 'Lastname']] = df['Name'].str.split(' ', 1, expand=True)
df.drop(columns=['Name'], inplace=True)

# 2. Chuyển đổi đơn vị cân nặng về kg (giả sử có đơn vị lb -> kg)
def convert_weight(weight):
    if isinstance(weight, str):
        if "lb" in weight:
            return round(float(weight.replace("lb", "")) * 0.453592, 2)
        elif "kg" in weight:
            return float(weight.replace("kg", ""))
    return weight

df["Weight"] = df["Weight"].apply(convert_weight)

# 3. Xóa dòng trống hoàn toàn
df.dropna(how="all", inplace=True)

# 4. Loại bỏ các dòng dữ liệu trùng lặp
df.drop_duplicates(inplace=True)

# 5. Xử lý lỗi non-ASCII
df = df.applymap(lambda x: x if isinstance(x, str) and x.isascii() else "warning")

# 6. Xử lý giá trị bị thiếu (Missing Values)
# Thống kê dữ liệu thiếu trên Age và Weight
print("\nTỉ lệ dữ liệu thiếu trên Age và Weight:")
print(df[["Age", "Weight"]].isnull().sum())

# Thay thế Age bằng giá trị trung bình của cột
df["Age"].fillna(df["Age"].mean(), inplace=True)

# Xóa dòng nếu cả Age và Weight đều bị thiếu
df.dropna(subset=["Age", "Weight"], how="all", inplace=True)

# 7. Tách cột `m0006` thành 3 cột: PulseRate, Sex, Time
df["Sex"] = df["m0006"].str[0]
df["Time"] = df["m0006"].str[1:]
df["PulseRate"] = pd.to_numeric(df["m0006"].str.extract("(\d+)", expand=False), errors='coerce')
df.drop(columns=["m0006"], inplace=True)

# 8. Reindex lại dữ liệu
df.reset_index(drop=True, inplace=True)

# 9. Lưu trữ dữ liệu sạch
df.to_csv("patient_heart_rate_clean.csv", index=False)

print("\nDữ liệu sau khi làm sạch và lưu thành công!")
print(df.head())


Dữ liệu ban đầu:
   Id      Name  Age Weight m0006 m0612 ...
0   1  John Doe   25  70kg  m120  m110 ...
1   2  Jane Roe   30  154lb f130  f115 ...
...

Tỉ lệ dữ liệu thiếu trên Age và Weight:
Age       3
Weight    5

Dữ liệu sau khi làm sạch và lưu thành công!
   Id  Age  Weight Firstname Lastname Sex  Time  PulseRate
0   1   25   70.0     John      Doe    m   120      120
1   2   30   69.9     Jane      Roe    f   130      130
...
